In [1]:
from pyspark.sql.types import StringType
import pyspark.sql.functions as f
import pandas as pd
from collections import Counter

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1551986734745_0003,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [24]:
a = list('A') * 5
b = list('B') * 2
c = list('C') * 3
d = list('D') * 2
l = a + b + c + d
l

['A', 'A', 'A', 'A', 'A', 'B', 'B', 'C', 'C', 'C', 'D', 'D']

In [31]:
races = ['w', 'w', 'b', 'u', 'w', 'b', 'u', 'u', 'u', 'w', 'u', 'u']

In [32]:
data = {"guid": l, "race": races}

In [33]:
df = pd.DataFrame(data)

In [34]:
df = spark.createDataFrame(df)

In [35]:
df.show()

+----+----+
|guid|race|
+----+----+
|   A|   w|
|   A|   w|
|   A|   b|
|   A|   u|
|   A|   w|
|   B|   b|
|   B|   u|
|   C|   u|
|   C|   u|
|   C|   w|
|   D|   u|
|   D|   u|
+----+----+

In [36]:
df.groupBy("guid").agg(f.collect_list(f.col("race")).alias('race_list')).show()

+----+---------------+
|guid|      race_list|
+----+---------------+
|   B|         [b, u]|
|   D|         [u, u]|
|   C|      [u, u, w]|
|   A|[w, w, b, u, w]|
+----+---------------+

In [48]:
@f.udf(StringType())
def get_race(col):
    # change function to address ties and unknown as top most common
    race = Counter(col).most_common(7)[0][0] # 7 race categories
    return race

In [45]:
# test function
print(races)
d = Counter(races)
print(d)
d.most_common(7)[0][0] # get top most common -- still need to address ties and unknown as top most common

['w', 'w', 'b', 'u', 'w', 'b', 'u', 'u', 'u', 'w', 'u', 'u']
Counter({'u': 6, 'w': 4, 'b': 2})
'u'

In [46]:
df\
    .groupBy("guid").agg(f.collect_list(f.col("race")).alias('race_list'))\
    .withColumn("race_census", get_race(f.col('race_list')))\
    .show()

+----+---------------+-----------+
|guid|      race_list|race_census|
+----+---------------+-----------+
|   B|         [b, u]|          b|
|   D|         [u, u]|          u|
|   C|      [u, u, w]|          u|
|   A|[w, w, b, u, w]|          w|
+----+---------------+-----------+

In [47]:
# Note there is only one data shuffle needed (group by Exchange) and the data never gets bigger
df\
    .groupBy("guid").agg(f.collect_list(f.col("race")).alias('race_list'))\
    .withColumn("race_census", get_race(f.col('race_list')))\
    .explain()

== Physical Plan ==
*(1) Project [guid#76, race_list#129, pythonUDF0#137 AS race_census#133]
+- BatchEvalPython [get_race(race_list#129)], [guid#76, race_list#129, pythonUDF0#137]
   +- ObjectHashAggregate(keys=[guid#76], functions=[collect_list(race#77, 0, 0)])
      +- Exchange hashpartitioning(guid#76, 200)
         +- ObjectHashAggregate(keys=[guid#76], functions=[partial_collect_list(race#77, 0, 0)])
            +- Scan ExistingRDD[guid#76,race#77]